In [ ]:
!pip install pymilvus

In [ ]:
!pip install langchain-community

In [ ]:
from langchain_community.vectorstores.milvus import Milvus

In [ ]:
!pip install milvus #pymilvus is the python sdk


In [ ]:
from milvus import default_server
default_server.start()

In [ ]:
from pymilvus import connections
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

In [ ]:
!pip install transformers accelerate einops langchain sentence-transformers

In [ ]:
!pip install langchain-community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.milvus import Milvus
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain_community.llms import Ollama
import os
import torch
import transformers

In [ ]:
# Membuat direktori
os.mkdir("/content/Data")

In [ ]:
#membaca dataset di direktori
document=[]
path = '/content/Data'
for file in os.listdir(path):
  file_path = os.path.join(path,file)
  loader = PyPDFLoader(file_path)
  document_load = loader.load()
  document.append(document_load)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding=HuggingFaceEmbeddings(
    model_name = embedding_model_name
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20
)

In [ ]:
split_docs = []
for doc in document:
    doc_chunks = text_splitter.split_documents(doc)
    split_docs.extend(doc_chunks)

In [ ]:
vector_db = Milvus.from_documents(
    split_docs,
    embedding,
)

In [ ]:
from google.colab import userdata
huggingfacehub_api_key = userdata.get('HUGGINGFACE_API_KEY')

In [ ]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_key,
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={"temperature":0.1}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:
from langchain.prompts import ChatPromptTemplate

template="""youe name is AI. You are an assistant to answering the question that user gave
question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser = StrOutputParser()

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
rag_chain=(
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
for chunk in rag_chain.stream("apa itu machine learning Menurut Shalev-Shwartz dan Ben-David?"):
    print(chunk, end="", flush=True)

Human: youe name is AI. You are an assistant to answering the question that user gave
question: apa itu machine learning Menurut Shalev-Shwartz dan Ben-David?
Context: [Document(page_content='Menjadi salah  satu teknologi baru yang  paling kuat, machine learning memainkan peran utama dalam \nkecerdasan buatan dan interaksi manusia. Machine Learning  adalah alat inovatif yang digunakan untuk \nmemerangi kanker, perubahan iklim, dan bahkan terorisme  [2]. Hal ini merupakan sebuah  inovasi  baru \nuntuk di segala bidang . Ak ibatnya, machine learning  membantu komputer menemukan informasi', metadata={'source': '/content/Data/ML.pdf', 'page': 0, 'pk': 449972605047801318}), Document(page_content='Menjadi salah  satu teknologi baru yang  paling kuat, machine learning memainkan peran utama dalam \nkecerdasan buatan dan interaksi manusia. Machine Learning  adalah alat inovatif yang digunakan untuk \nmemerangi kanker, perubahan iklim, dan bahkan terorisme  [2]. Hal ini merupakan sebuah  inovasi

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
#!pip install bitsandbytes

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
)

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024)

In [ ]:
answer=pipe(["apa itu machine learning Menurut Shalev-Shwartz dan Ben-David?"])


print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
from langchain.llms import HuggingFacePipeline